In [1]:
import torch
from sklearn.datasets import fetch_california_housing, make_friedman1
from src.mssp import MSSP

In [2]:
data = fetch_california_housing()
X = torch.tensor(data.data, dtype=torch.double)
y = torch.tensor(data.target, dtype=torch.double)

In [3]:
# X, y = make_friedman1(n_samples=5000, n_features=10, noise=1.0, random_state=0)
# X = torch.tensor(X, dtype=torch.float32)
# y = torch.tensor(y, dtype=torch.float32)

In [4]:
X.shape, y.shape

(torch.Size([20640, 8]), torch.Size([20640]))

In [5]:
# model = MSSP(n_best=115, loss_fn="mse", random_seed=69, epochs=10, diversity_ratio=0.15, pow_cross=True) #ggets to 0.41
model = MSSP(n_best=120, loss_fn="mse", random_seed=42, epochs=10, diversity_ratio=0.8, pow_cross=True)

In [6]:
model.fit(X, y)
# rmse:
    # random seed 42, allow diversity (25%):
        # Best loss:  
        # 2.2263708114624023 after training for 299 epochs
    # random seed 69, allow diversity (15%):
        # Best loss: 2.264385223388672 after training for 63 epochs
    # random seed 69, allow diversity (25%):
        # Best loss: 2.165372610092163 after training for 299 epochs
    # random seed 69, allow diversity (35%):
        # Best loss: 2.187122344970703 after training for 299 epochs
    # no diversity:
        # Best loss: 2.3074421882629395 after training for 58 epochs

torch.Size([20640, 64])
loss (mse): 0.6123 epoch: 0 , time: 0.94s
torch.Size([20640, 119])
loss (mse): 0.4733 epoch: 1 , time: 3.17s
torch.Size([20640, 119])
loss (mse): 0.4610 epoch: 2 , time: 2.42s
torch.Size([20640, 119])
loss (mse): 0.4445 epoch: 3 , time: 2.52s
torch.Size([20640, 119])
loss (mse): 0.4381 epoch: 4 , time: 2.60s
torch.Size([20640, 119])
loss (mse): 0.4335 epoch: 5 , time: 2.59s
torch.Size([20640, 119])
loss (mse): 0.4274 epoch: 6 , time: 2.60s
torch.Size([20640, 119])
loss (mse): 0.4233 epoch: 7 , time: 2.65s
torch.Size([20640, 119])
loss (mse): 0.4225 epoch: 8 , time: 2.49s
torch.Size([20640, 119])
loss (mse): 0.4210 epoch: 9 , time: 2.38s
Best loss: 0.42096826434135437 after training for 9 epochs


In [7]:
hist = []
for h in model.history:
    # if h['type'] != 'best_selection':
    if h['epoch'] != 0:
        continue
    hist.append(h)
    for k, v in h.items():
        print(k, v)
        if type(v) == type(torch.tensor):
            print(len(v))
    print("-"*100)

mask tensor([ 453,   40,  455,  223,   42,  425,  422,  426,  420,  427,  397,  429,
          12,    9,   13,    7,  423,   14,  421,  431,  225,   16,   10, 1118,
        1805, 1231, 1727,  525,  755, 1754,  669,  590,  654,  811, 1560, 1416,
         984, 1747, 1866,  254,  637,  231, 1883, 1080,  440, 1375,  275, 1533,
        1139, 1006, 1400,  471, 1249,  346,  189,   17,  992, 1404, 1928, 1200,
        1220,  727, 1414, 1130,  452,  700, 1230,  848, 1293, 1348,  267,  198,
        1022, 1475, 1225, 1776, 1076,  164,  279,  629,  707, 1182,   50, 1894,
        1593,  365, 1811,   26,  102, 1600,  290, 1117, 1491,  150, 1638, 1257,
        1633,  560,  777,  348,  730, 1067, 1089, 1992, 1549, 1023, 1471,  175,
        1781, 1382, 1670, 1982, 1673, 1451, 1402,  385, 1855, 1846, 1208])
epoch 0
coef tensor([[ 9.9529e-01,  9.6510e-01],
        [ 9.9605e-01,  9.7154e-01],
        [ 1.0018e+00,  8.1133e-01],
        [ 1.4557e+00,  9.4544e-01],
        [ 1.0022e+00,  8.1480e-01],
       

In [8]:
lin = [h for h in hist if h['epoch'] == 5 and h['type'] == 'lin_cross_selection'][0]
pow = [h for h in hist if h['epoch'] == 5 and h['type'] == 'pow_cross_selection'][0]
best = [h for h in hist if h['epoch'] == 5 and h['type'] == 'best_selection'][0]['mask'][5]

coef = torch.cat([lin['coef'], pow['coef']], dim=0)
intercept = torch.cat([lin['intercept'], pow['intercept']], dim=0)
i = torch.cat([lin['i'], pow['i']], dim=0)
j = torch.cat([lin['j'], pow['j']], dim=0)

IndexError: list index out of range

In [ ]:
coef[best]

In [ ]:
model._build_model(5)

In [ ]:
torch.tensor([1,5,7,3,5], dtype=float) @ torch.tensor([0,0,0.5,0,0], dtype=float)

In [ ]:
arr = torch.randn(200,5)

In [ ]:
torch.vmap(lambda row: torch.tensor([1,4,5,6,7]))(arr)